# 引用需要的函式庫

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split

import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import ImageFolder


import numpy as np
import pandas as pd
from PIL import Image

# 自定義資料集
讀取CSV檔中的MNIST資料集，每一列代表一個圖片的類別與784個畫素值  
將784個畫素重新排列為28*28的二維陣列，並轉換為Pillow image便於後續的資料前處理  

In [2]:
class MNISTDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file) # read data from csv file as a pandas dataframe
        self.transform = transform # initial transfrom

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # transfer data from list to Pillow image
        image = Image.fromarray(self.data_frame.iloc[idx, 1:].values.reshape(28, 28).astype(np.uint8))
        
        label = int(self.data_frame.iloc[idx, 0])

        if self.transform:
            image = self.transform(image)

        return image, label

# 資料前處理
定義適用於訓練集與測試集的transform分別進行各自的資料前處理流程

In [3]:
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=90),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4850, 0.4560, 0.4060], std=[0.2290, 0.2240, 0.2250])
])

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4850, 0.4560, 0.4060], std=[0.2290, 0.2240, 0.2250])
])

# 建立訓練集、驗證集、測試集
由於MNIST資料集未提供獨立的驗證集，因此將一半的測試集拆分出來做為驗證集  
驗證集可以監控模型是否overfitting，同時作為保存最佳模型的標準

In [4]:
train_path = 'mnist/mnist_train.csv'
test_path = 'mnist/mnist_test.csv'

train_data = MNISTDataset(csv_file=train_path, transform=train_transform)
test_data = MNISTDataset(csv_file=test_path, transform=test_transform)
testLen = int(len(test_data) * 0.5)
valLen = len(test_data) - testLen
test_data, val_data = random_split(test_data, [testLen, valLen])

# 建立DataLoader
可決定batch size、以及資料集載入時是否打亂

In [5]:
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
val_loader = DataLoader(val_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)

# 訓練用硬體
決定使用GPU或是CPU進行訓練  
範例優先選用GPU，若GPU不可用則使用CPU

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# 選擇Model、Loss Function以及Optimizer
修改最後的全連階層使其適應資料集的類別數量  
設定完成後會將model移至GPU/CPU

In [7]:
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), weight_decay=0.001, lr=0.0001)

model.to(device)

d:\miniconda3\envs\summercourse\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\miniconda3\envs\summercourse\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# 訓練過程
範例訓練過程會歷遍資料集共20輪  
每一輪都會計算training與validation的loss與accuracy  
以validation accuracy為標準保存最佳的模型參數

In [8]:
num_epochs = 20
best_accuracy = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = correct_train / total_train

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

        val_loss = running_val_loss / len(val_loader.dataset)
        val_accuracy = correct_val / total_val

        if val_accuracy > best_accuracy:
            # Save model weights
            torch.save(model.state_dict(), 'best_model.pth')
            best_accuracy = val_accuracy

        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/20, Training Loss: 1.0478, Training Accuracy: 0.6540
Validation Loss: 0.5730, Validation Accuracy: 0.8206
Epoch 2/20, Training Loss: 0.4921, Training Accuracy: 0.8426
Validation Loss: 0.4942, Validation Accuracy: 0.8380
Epoch 3/20, Training Loss: 0.3884, Training Accuracy: 0.8730
Validation Loss: 0.3654, Validation Accuracy: 0.8774
Epoch 4/20, Training Loss: 0.3391, Training Accuracy: 0.8887
Validation Loss: 0.3044, Validation Accuracy: 0.8984
Epoch 5/20, Training Loss: 0.3170, Training Accuracy: 0.8965
Validation Loss: 0.2890, Validation Accuracy: 0.9068
Epoch 6/20, Training Loss: 0.2953, Training Accuracy: 0.9033
Validation Loss: 0.3008, Validation Accuracy: 0.8980
Epoch 7/20, Training Loss: 0.2735, Training Accuracy: 0.9096
Validation Loss: 0.2206, Validation Accuracy: 0.9256
Epoch 8/20, Training Loss: 0.2567, Training Accuracy: 0.9140
Validation Loss: 0.1914, Validation Accuracy: 0.9426
Epoch 9/20, Training Loss: 0.2499, Training Accuracy: 0.9165
Validation Loss: 0.2146, Va

# 測試模型
載入最佳的模型參數  
在testing set測試模型並計算loss與accuracy

In [9]:
model.load_state_dict(torch.load('best_model.pth'))

model.eval()  # 设置模型为评估模式
running_test_loss = 0.0
correct_test = 0
total_test = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_loss = running_test_loss / len(test_loader.dataset)
test_accuracy = correct_test / total_test

print(f'Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}')

C:\Users\mingyiao\AppData\Local\Temp\ipykernel_32292\2173399408.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Testing Loss: 0.1535, Testing Accuracy: 0.9490
